# Post 성수동 찾기: 하이브리드 모델
### Model 2 (서울 전체) + 성수동 특성 가중치 결합

## STEP 0: 데이터 로드 및 전처리

In [10]:
import pandas as pd
import glob
import numpy as np
import os

# [환경설정] 절대 경로 설정
base_path = '/teamspace/studios/this_studio'
data_path = os.path.join(base_path, 'raw_data')

# 1. 선생님이 확정한 업종 및 컬럼 리스트
target_sectors = [
    '섬유제품', '완구', '운동/경기_용품', '화장품', '문구', '서적', 
    '시계및귀금속', '안경', '일반의류', '편의점', '노래방', '미용실', 
    '당구장', '커피-음료', '호프-간이주점', '분식전문점', '치킨전문점', 
    '패스트푸드점', '제과점', '양식음식점', '중식음식점', '한식음식점'
]

cols_to_keep = [
    '기준_년분기_코드', '행정동_코드', '행정동_코드_명', '서비스_업종_코드_명', 
    '당월_매출_금액', '주중_매출_금액', '주말_매출_금액', 
    '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액'
]

# 2. 연도별 매출 데이터 통합
print("진행 중: 연도별 추정매출 통합...")
sales_files = sorted(glob.glob(os.path.join(data_path, '매출_*.csv')))
sales_list = []

for f in sales_files:
    df = pd.read_csv(f, usecols=cols_to_keep, encoding='cp949')
    df = df[df['서비스_업종_코드_명'].isin(target_sectors)]
    sales_list.append(df)

df_sales = pd.concat(sales_list, ignore_index=True)

# 3. 주말 및 MZ 매출 가중치 부여
print("진행 중: 주말 및 MZ 매출 가중치 적용...")
df_sales['주말_매출_가중'] = df_sales['주말_매출_금액'] * 1.5
df_sales['MZ_매출_가중'] = (
    df_sales['연령대_10_매출_금액'] * 1.2 + 
    df_sales['연령대_20_매출_금액'] * 1.2 + 
    df_sales['연령대_30_매출_금액'] * 1.2
)

df_sales['가중_총매출'] = (
    df_sales['당월_매출_금액'] * 0.3 +
    df_sales['주말_매출_가중'] * 0.3 +
    df_sales['MZ_매출_가중'] * 0.4
)

# 4. 단일 파일 데이터 로드
print("진행 중: 기타 테이블 병합...")

df_pop = pd.read_csv(os.path.join(data_path, '유동인구.csv'), encoding='cp949')
df_resident = pd.read_csv(os.path.join(data_path, '상주인구.csv'), encoding='cp949')
df_change = pd.read_csv(os.path.join(data_path, '상권변화지표.csv'), encoding='cp949')
df_facility = pd.read_csv(os.path.join(data_path, '집객시설.csv'), encoding='cp949')

# 유동인구
df_pop = df_pop[['기준_년분기_코드', '행정동_코드', '총_유동인구_수', '연령대_10_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수']]
df_pop['MZ_유동인구'] = df_pop['연령대_10_유동인구_수'] + df_pop['연령대_20_유동인구_수'] + df_pop['연령대_30_유동인구_수']

# 상주인구
df_resident = df_resident[['기준_년분기_코드', '행정동_코드', '총_상주인구_수', '총_가구_수']]

# 집객시설
df_facility = df_facility[['기준_년분기_코드', '행정동_코드', '집객시설_수', '지하철_역_수']]

# 상권변화지표
df_change = df_change[['기준_년분기_코드', '행정동_코드', '상권_변화_지표_명', '운영_영업_개월_평균']]
mapping = {'다이나믹': 4, '상권확장': 3, '정체': 2, '상권축소': 1}
df_change['상권지표_점수'] = df_change['상권_변화_지표_명'].map(mapping).fillna(0)

# 5. 최종 Merge
final_df = df_sales.merge(df_pop, on=['기준_년분기_코드', '행정동_코드'], how='left')
final_df = final_df.merge(df_resident, on=['기준_년분기_코드', '행정동_코드'], how='left')
final_df = final_df.merge(df_change, on=['기준_년분기_코드', '행정동_코드'], how='left')
final_df = final_df.merge(df_facility, on=['기준_년분기_코드', '행정동_코드'], how='left')

final_df.fillna(0, inplace=True)
print(f"\n✅ 데이터 준비 완료! 총 {len(final_df):,}개 행")

진행 중: 연도별 추정매출 통합...
진행 중: 주말 및 MZ 매출 가중치 적용...
진행 중: 기타 테이블 병합...

✅ 데이터 준비 완료! 총 188,294개 행


## STEP 1: 데이터 검증

In [11]:
print(f"매출 데이터 원본 행 수: {len(df_sales):,}")
print(f"최종 병합 데이터 행 수: {len(final_df):,}")

if len(df_sales) == len(final_df):
    print("✅ 성공: 데이터 누락이나 중복 없이 완벽하게 병합되었습니다.")
else:
    print("⚠️ 주의: 행 개수가 다릅니다.")

매출 데이터 원본 행 수: 188,294
최종 병합 데이터 행 수: 188,294
✅ 성공: 데이터 누락이나 중복 없이 완벽하게 병합되었습니다.


## STEP 2: 행정동별 그룹화 및 파생변수 생성

In [12]:
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

# 행정동명 정리
df_analysis = final_df.copy()
df_analysis['행정동_코드_명'] = df_analysis['행정동_코드_명'].str.replace(r'[^가-힣0-9]', '', regex=True)

# 행정동별/분기별 그룹화
df_grouped = df_analysis.groupby(['기준_년분기_코드', '행정동_코드_명']).agg({
    '가중_총매출': 'sum',
    'MZ_유동인구': 'sum',
    '총_유동인구_수': 'sum',
    '총_상주인구_수': 'mean',
    '주말_매출_금액': 'sum',
    '연령대_10_매출_금액': 'sum',
    '연령대_20_매출_금액': 'sum',
    '연령대_30_매출_금액': 'sum',
    '집객시설_수': 'max', 
    '지하철_역_수': 'max',
    '운영_영업_개월_평균': 'mean'
}).reset_index().sort_values(['행정동_코드_명', '기준_년분기_코드'])

# MZ 유동인구 증가율 (종속변수)
df_grouped['MZ_유동_증가율'] = df_grouped.groupby('행정동_코드_명')['MZ_유동인구'].pct_change()
df_grouped = df_grouped.replace([np.inf, -np.inf], np.nan).dropna(subset=['MZ_유동_증가율'])

# 파생 변수 생성
df_grouped['상권_에너지_지수'] = df_grouped['가중_총매출'] * df_grouped['집객시설_수']
df_grouped['상권_유입_강도'] = df_grouped['총_유동인구_수'] / (df_grouped['총_상주인구_수'] + 1)
df_grouped['주말_매출_비중'] = df_grouped['주말_매출_금액'] / (df_grouped['가중_총매출'] + 1)

# MZ 매출 비중 (20대 + 30대만, 10대 제외)
total_mz_sales = df_grouped['연령대_20_매출_금액'] + df_grouped['연령대_30_매출_금액']
df_grouped['MZ_매출_비중'] = total_mz_sales / (df_grouped['가중_총매출'] + 1)

print(f"전체 데이터 행 수: {len(df_grouped):,}")
print(f"전체 행정동 수: {df_grouped['행정동_코드_명'].nunique()}개")

전체 데이터 행 수: 10,992
전체 행정동 수: 424개


## STEP 3: Model 2 회귀분석 (서울 전체)

In [13]:
print("=" * 60)
print("[STEP 3] Model 2: 서울 전체 데이터 회귀분석")
print("=" * 60)

# 분석용 변수 (Model 2)
cols_model2 = ['집객시설_수', '지하철_역_수', '운영_영업_개월_평균', '상권_에너지_지수', '주말_매출_비중', 'MZ_매출_비중']

# 분석용 변수 (성수동 - 상권_유입_강도 포함)
cols_seongsu = ['집객시설_수', '지하철_역_수', '운영_영업_개월_평균', '상권_에너지_지수', '상권_유입_강도', '주말_매출_비중']

# 전체 변수 통합 (정규화용)
cols_all = list(set(cols_model2 + cols_seongsu))

# 정규화
scaler = MinMaxScaler()
df_scaled = df_grouped.copy()
df_scaled[cols_all] = scaler.fit_transform(df_grouped[cols_all])

# Model 2 OLS 회귀분석
X_model2 = sm.add_constant(df_scaled[cols_model2])
Y_model2 = df_scaled['MZ_유동_증가율']

model2 = sm.OLS(Y_model2, X_model2).fit()
model2_weights = model2.params

print("\n=== Model 2 회귀분석 결과 ===")
print(model2.summary())
print("\n★ Model 2 가중치:")
print(model2_weights)

[STEP 3] Model 2: 서울 전체 데이터 회귀분석

=== Model 2 회귀분석 결과 ===
                            OLS Regression Results                            
Dep. Variable:              MZ_유동_증가율   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     5.414
Date:                Wed, 14 Jan 2026   Prob (F-statistic):           1.34e-05
Time:                        15:14:58   Log-Likelihood:                 13227.
No. Observations:               10992   AIC:                        -2.644e+04
Df Residuals:                   10985   BIC:                        -2.639e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

## STEP 4: 성수동 특성 가중치 추출

In [14]:
print("=" * 60)
print("[STEP 4] 성수동 특성 가중치 추출")
print("=" * 60)

# 성수동 데이터 필터링
seongsu_df = df_scaled[df_scaled['행정동_코드_명'].str.contains('성수')].copy()

print(f"\n성수동 데이터 개수: {len(seongsu_df)}개")
print(f"성수동 행정동 목록: {seongsu_df['행정동_코드_명'].unique()}")

# 성수동 회귀분석
X_seongsu = sm.add_constant(seongsu_df[cols_seongsu])
Y_seongsu = seongsu_df['MZ_유동_증가율']

model_seongsu = sm.OLS(Y_seongsu, X_seongsu).fit()
seongsu_weights = model_seongsu.params

print("\n=== 성수동 회귀분석 결과 (참고용) ===")
print(model_seongsu.summary())
print("\n★ 성수동 특성 가중치:")
print(seongsu_weights)

# 성수동 프로파일 출력
seongsu_profile = df_grouped[df_grouped['행정동_코드_명'].str.contains('성수')].mean(numeric_only=True)
print("\n★ 성수동 프로파일 (평균값):")
print(f"  - 상권_에너지_지수: {seongsu_profile['상권_에너지_지수']:.2e}")
print(f"  - 상권_유입_강도: {seongsu_profile['상권_유입_강도']:.2f}")
print(f"  - 주말_매출_비중: {seongsu_profile['주말_매출_비중']:.4f}")
print(f"  - MZ_매출_비중: {seongsu_profile['MZ_매출_비중']:.4f}")
print(f"  - 운영_영업_개월_평균: {seongsu_profile['운영_영업_개월_평균']:.1f}개월")

[STEP 4] 성수동 특성 가중치 추출

성수동 데이터 개수: 104개
성수동 행정동 목록: ['성수1가1동' '성수1가2동' '성수2가1동' '성수2가3동']

=== 성수동 회귀분석 결과 (참고용) ===
                            OLS Regression Results                            
Dep. Variable:              MZ_유동_증가율   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.9249
Date:                Wed, 14 Jan 2026   Prob (F-statistic):              0.481
Time:                        15:14:58   Log-Likelihood:                 158.95
No. Observations:                 104   AIC:                            -303.9
Df Residuals:                      97   BIC:                            -285.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
------------

## STEP 5: 하이브리드 점수 계산

In [15]:
print("=" * 60)
print("[STEP 5] 하이브리드 점수 계산")
print("=" * 60)

# 1. Model 2 점수 (통계적으로 유의미한 변수만 사용)
df_scaled['Model2_점수'] = (
    df_scaled['상권_에너지_지수'] * model2_weights['상권_에너지_지수'] +
    df_scaled['운영_영업_개월_평균'] * model2_weights['운영_영업_개월_평균'] +
    df_scaled['집객시설_수'] * model2_weights['집객시설_수'] +
    df_scaled['주말_매출_비중'] * model2_weights['주말_매출_비중'] +
    df_scaled['MZ_매출_비중'] * model2_weights['MZ_매출_비중']
)

# 2. 성수동 유사도 점수
df_scaled['성수_유사도_점수'] = (
    seongsu_weights['const'] +
    df_scaled['집객시설_수'] * seongsu_weights['집객시설_수'] +
    df_scaled['지하철_역_수'] * seongsu_weights['지하철_역_수'] +
    df_scaled['운영_영업_개월_평균'] * seongsu_weights['운영_영업_개월_평균'] +
    df_scaled['상권_에너지_지수'] * seongsu_weights['상권_에너지_지수'] +
    df_scaled['상권_유입_강도'] * seongsu_weights['상권_유입_강도'] +
    df_scaled['주말_매출_비중'] * seongsu_weights['주말_매출_비중']
)

# 3. 하이브리드 점수 (Model 2: 70% + 성수 유사도: 30%)
WEIGHT_MODEL2 = 0.7
WEIGHT_SEONGSU = 0.3

df_scaled['하이브리드_점수'] = (
    df_scaled['Model2_점수'] * WEIGHT_MODEL2 + 
    df_scaled['성수_유사도_점수'] * WEIGHT_SEONGSU
)

print(f"\n가중치 설정: Model 2 ({WEIGHT_MODEL2*100:.0f}%) + 성수 유사도 ({WEIGHT_SEONGSU*100:.0f}%)")
print(f"\n점수 범위:")
print(f"  - Model2_점수: {df_scaled['Model2_점수'].min():.4f} ~ {df_scaled['Model2_점수'].max():.4f}")
print(f"  - 성수_유사도_점수: {df_scaled['성수_유사도_점수'].min():.4f} ~ {df_scaled['성수_유사도_점수'].max():.4f}")
print(f"  - 하이브리드_점수: {df_scaled['하이브리드_점수'].min():.4f} ~ {df_scaled['하이브리드_점수'].max():.4f}")

[STEP 5] 하이브리드 점수 계산

가중치 설정: Model 2 (90%) + 성수 유사도 (10%)

점수 범위:
  - Model2_점수: -0.0569 ~ -0.0077
  - 성수_유사도_점수: -0.6575 ~ 1721.3067
  - 하이브리드_점수: -0.0730 ~ 172.0986


## STEP 6: Post 성수동 최종 랭킹

In [16]:
print("=" * 60)
print("[STEP 6] Post 성수동 최종 랭킹")
print("=" * 60)

# 행정동별 평균 점수 계산
ranking_hybrid = df_scaled.groupby('행정동_코드_명').agg({
    'Model2_점수': 'mean',
    '성수_유사도_점수': 'mean',
    '하이브리드_점수': 'mean',
    'MZ_매출_비중': 'mean',
    '상권_유입_강도': 'mean',
    '주말_매출_비중': 'mean'
}).reset_index()

# 전체 랭킹 (성수동 포함)
ranking_all = ranking_hybrid.sort_values('하이브리드_점수', ascending=False).reset_index(drop=True)
ranking_all.insert(0, 'Rank', range(1, len(ranking_all) + 1))

# 성수동 현재 순위 확인
print("\n★ 현재 성수동 순위 (검증용):")
seongsu_rank = ranking_all[ranking_all['행정동_코드_명'].str.contains('성수')]
print(seongsu_rank[['Rank', '행정동_코드_명', '하이브리드_점수', 'Model2_점수', '성수_유사도_점수']].to_string(index=False))

# Post 성수동 랭킹 (성수동 제외)
ranking_post_seongsu = ranking_all[~ranking_all['행정동_코드_명'].str.contains('성수')].copy()
ranking_post_seongsu['Rank'] = range(1, len(ranking_post_seongsu) + 1)

print("\n" + "=" * 60)
print("★★★ Post 성수동 TOP 30 ★★★")
print("=" * 60)
print(ranking_post_seongsu[['Rank', '행정동_코드_명', '하이브리드_점수', 'Model2_점수', '성수_유사도_점수']].head(30).to_string(index=False))

[STEP 6] Post 성수동 최종 랭킹

★ 현재 성수동 순위 (검증용):
 Rank 행정동_코드_명  하이브리드_점수  Model2_점수  성수_유사도_점수
   29   성수1가2동 -0.028918  -0.033907   0.015979
   71   성수2가3동 -0.030952  -0.035433   0.009374
  147   성수1가1동 -0.032902  -0.036391  -0.001508
  292   성수2가1동 -0.035095  -0.039391   0.003568

★★★ Post 성수동 TOP 30 ★★★
 Rank 행정동_코드_명  하이브리드_점수  Model2_점수  성수_유사도_점수
    1     수유3동 12.778119  -0.039579 128.137400
    2      번1동  8.605447  -0.040025  86.414695
    3     수유1동  6.678036  -0.039535  67.136178
    4     수유2동  5.085230  -0.037374  51.188665
    5      번3동  2.688880  -0.037165  27.223280
    6      번2동  1.732553  -0.038323  17.670432
    7     을지로동 -0.009339  -0.042454   0.288696
    8     둔촌1동 -0.015017  -0.023174   0.058399
    9     북아현동 -0.021985  -0.039798   0.138328
   10     개포1동 -0.025240  -0.027777  -0.002400
   11      소공동 -0.025423  -0.038341   0.090832
   12      신당동 -0.025611  -0.033606   0.046340
   13   종로56가동 -0.025991  -0.042886   0.126056
   14       항동 -0.026015  -0.029594   

## STEP 7: 결과 저장

In [17]:
# CSV 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f'Post_성수동_하이브리드_랭킹_{timestamp}.csv'
output_path = os.path.join(base_path, output_filename)

# 저장할 컬럼 선택
save_cols = ['Rank', '행정동_코드_명', '하이브리드_점수', 'Model2_점수', '성수_유사도_점수', 'MZ_매출_비중', '상권_유입_강도', '주말_매출_비중']
ranking_post_seongsu[save_cols].head(50).to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n✅ 결과 저장 완료: {output_path}")
print(f"   저장된 행정동 수: 50개")


✅ 결과 저장 완료: /teamspace/studios/this_studio/Post_성수동_하이브리드_랭킹_20260114_151458.csv
   저장된 행정동 수: 50개


## STEP 8: 분석 요약 리포트

In [18]:
print("=" * 60)
print("[분석 요약 리포트]")
print("=" * 60)

print("""
★ 분석 방법론:

1. Model 2 (서울 전체 학습)
   - 데이터: 서울 전체 행정동 (10,992개 관측치)
   - 통계적 유의성: Prob(F) < 0.05 ✓
   - 유의미한 변수: 상권_에너지_지수, 운영_영업_개월_평균, 
                   MZ_매출_비중, 주말_매출_비중, 집객시설_수

2. 성수동 특성 가중치
   - 데이터: 성수동 4개 행정동만
   - 목적: 성수동만의 고유한 패턴 추출
   - 핵심 특성: 상권_유입_강도, 주말_매출_비중

3. 하이브리드 점수
   - 구성: Model 2 (70%) + 성수 유사도 (30%)
   - 이유: Model 2는 통계적으로 유의하므로 높은 가중치
          성수동 특성은 참고용으로 30% 반영

★ Post 성수동 선정 기준:
   - 하이브리드 점수가 높은 지역
   - 성수동과 유사한 상권 특성 보유
   - 아직 MZ 포화가 되지 않아 성장 잠재력 있는 지역
""")

# TOP 10 상세 분석
print("\n" + "=" * 60)
print("★ Post 성수동 TOP 10 상세 분석")
print("=" * 60)
top10 = ranking_post_seongsu.head(10)
for idx, row in top10.iterrows():
    print(f"\n{int(row['Rank'])}위: {row['행정동_코드_명']}")
    print(f"   - 하이브리드 점수: {row['하이브리드_점수']:.4f}")
    print(f"   - Model2 점수: {row['Model2_점수']:.4f}")
    print(f"   - 성수 유사도: {row['성수_유사도_점수']:.4f}")
    print(f"   - MZ 매출 비중: {row['MZ_매출_비중']:.4f}")

[분석 요약 리포트]

★ 분석 방법론:

1. Model 2 (서울 전체 학습)
   - 데이터: 서울 전체 행정동 (10,992개 관측치)
   - 통계적 유의성: Prob(F) < 0.05 ✓
   - 유의미한 변수: 상권_에너지_지수, 운영_영업_개월_평균, 
                   MZ_매출_비중, 주말_매출_비중, 집객시설_수

2. 성수동 특성 가중치
   - 데이터: 성수동 4개 행정동만
   - 목적: 성수동만의 고유한 패턴 추출
   - 핵심 특성: 상권_유입_강도, 주말_매출_비중

3. 하이브리드 점수
   - 구성: Model 2 (70%) + 성수 유사도 (30%)
   - 이유: Model 2는 통계적으로 유의하므로 높은 가중치
          성수동 특성은 참고용으로 30% 반영

★ Post 성수동 선정 기준:
   - 하이브리드 점수가 높은 지역
   - 성수동과 유사한 상권 특성 보유
   - 아직 MZ 포화가 되지 않아 성장 잠재력 있는 지역


★ Post 성수동 TOP 10 상세 분석

1위: 수유3동
   - 하이브리드 점수: 12.7781
   - Model2 점수: -0.0396
   - 성수 유사도: 128.1374
   - MZ 매출 비중: 0.6344

2위: 번1동
   - 하이브리드 점수: 8.6054
   - Model2 점수: -0.0400
   - 성수 유사도: 86.4147
   - MZ 매출 비중: 0.5816

3위: 수유1동
   - 하이브리드 점수: 6.6780
   - Model2 점수: -0.0395
   - 성수 유사도: 67.1362
   - MZ 매출 비중: 0.3698

4위: 수유2동
   - 하이브리드 점수: 5.0852
   - Model2 점수: -0.0374
   - 성수 유사도: 51.1887
   - MZ 매출 비중: 0.4618

5위: 번3동
   - 하이브리드 점수: 2.6889
   - Model2 점수: -0.0372
   - 성수 유사도: 27.2